<h1 align="center" style="color:green;font-size:40px;">The Art of Refactoring Your Existing Code</h1>  
<h2 align="center" style="color:gray;font-size:35px;margin: 0;">Mehadi Hasan Menon</h2>  
<h3 align="center" style="color:gray;font-size:30px;margin: 0;">AI Engineer</h3>
<h3 align="center" style="color:gray;font-size:30px;margin: 0;">Next Solution Lab</h3>
<h5 align="center" style="color:blue;font-size:25px;margin: 0;">menon@nextsolutionlab.com</h5>  

## Contents
1. What is refactoring
2. Why we need refactoring
3. What evil about not refactored code
4. Goal of refactoring
5. Refactoring rules we cover with concrete example
    - Extract method
    - Replace magic numbers with symbolic constant
    - Decompose conditionnal
    - Indroduce explaining variable
6. Conclution
7. Reference
8. Question and answer

## Refactoring
__Which one is looks better ?__
![Refactoring](images/refactoring.png)


## Refactoring | Defination
`Improve the desing of a program without altering it's behavior`

## Refactoring | Example
__Let we want to filter even numbers from a list__
```python
numbers = [1, 2, 3, 4, 5, 6, 7, 8, 9, 10]
# Solution - 1
even_numbers = []
for num in numbers:
    if num % 2 == 0:
        even_numbers.append(num)

# Solution - 2
# gen of python `flat is better then nested`
even_numbers = [num for num in numbers if num % 2 == 0]
```
`Both solution perform the same taks but in different ways`

# Why we need refactoring
- We read code more then we write
- There are may way to write code to implement one specefic behavior
- But every way is the the best way to read and understand the code better

![Image](images/what-evil-about-dead-code.jpg)

## Goal of refactoring
- Make software easier for a programmer to understand
- Correcting common patter that make the code harder to read

## Refactoring with concrete example

## Let's we have
- `Account` class
- `Drive` class, that manipulate `Account` class

In [1]:
class Account:
    '''defination of Account class'''
    # defins flags for 4 different types of account
    standard = 0
    budget = 1
    premium = 2
    premium_plus = 3
    
    def __init__(self, principal, rate, days_active, account_type):
        self.principal = principal
        self.rate = rate
        self.days_active = days_active
        self.account_type = account_type

class Drive:
    def calculate_fee(self, accounts: list) -> int:
        '''calculate fees for accounts'''
        total_fee = 0
        # iterate over all the accounts
        for account in accounts:
            # check account type
            if account.account_type == Account.premium or \
                account.account_type == Account.premium_plus:
                # calculate fee
                total_fee += 0.0125 * (account.principal * math.exp(
                    account.rate * (account.days_active / 365)) - account.principal)
        return total_fee

## Concrete example | Extract method


## What is the problem with above code ?


Perpose of calculation perform in line `25-29` is not clear
```python
total_fee += 0.0125 * (account.principal * math.exp(
    account.rate * (account.days_active / 365)) - account.principal)
```

## What was actual perpose of  line `25-29` calculation ?
`Interest calculation` to figure out how much interest was earn on the account
```python
total_fee += 0.0125 * (account.principal * math.exp(
    account.rate * (account.days_active / 365)) - account.principal)
```

## How do we refactor this for better understanding  the code ?
- Add a new methdo to `Account` class
- Use `extract method` rules

## Effect of`Extract Method` in line 25
- Add a method named `interest_earned` in `Account` class
- Remove interest calculation from `calculate_fee` method 
- Use `interest_earned` method inside `calculate_fee` method
```python
total_fee += 0.0125 * (account.principal * math.exp(
    account.rate * (account.days_active / 365)) - account.principal)
```
__Change to__
```python
def interest_earned(self):
    return (self.principal * math.exp(
        self.rate * (self.days_active / 365))) - self.principal
```

In [2]:
class Account:
    '''defination of Account class'''
    # defins flags for 4 different types of account
    standard = 0
    budget = 1
    premium = 2
    premium_plus = 3
    
    def __init__(self, principal, rate, days_active, account_type):
        self.principal = principal
        self.rate = rate
        self.days_active = days_active
        self.account_type = account_type
    
    def interest_earned(self):
        return (self.principal * math.exp(
            self.rate * (self.days_active / 365))) - self.principal
class Drive:
    def calculate_fee(self, accounts: list) -> int:
        '''calculate fees for accounts'''
        total_fee = 0
        for account in accounts:
            if account.account_type == Account.premium or \
                account.account_type == Account.premium_plus:
                # use `Account` class `interest_earned` method
                total_fee += 0.0125 * account.interest_earned()
        return total_fee

## Concrete example | Repace magic number with symbolic constant

## Now what the number `0.0125 in line 24` ? 
```python
total_fee += 0.0125 * account.interest_earned()
```
Percentage of `earned commision fee` by account holder

## Let's assign a meaningfull name to this magic number
```python
broker_fee_percent = 0.0125
```
__And perform__
```python
total_fee += Drive.broker_fee_percent * account.interest_earned()
```

In [3]:
class Account:
    '''defination of Account class'''
    # defins flags for 4 different types of account
    standard = 0
    budget = 1
    premium = 2
    premium_plus = 3
    
    def __init__(self, principal, rate, days_active, account_type):
        self.principal = principal
        self.rate = rate
        self.days_active = days_active
        self.account_type = account_type
    
    def interest_earned(self):
        return (self.principal * math.exp(
            self.rate * (self.days_active / 365))) - self.principal
class Drive:
    broker_fee_percent = 0.0125
    
    def calculate_fee(self, accounts: list) -> int:
        '''calculate fees for accounts'''
        total_fee = 0
        for account in accounts:
            if account.account_type == Account.premium or \
                account.account_type == Account.premium_plus:
                total_fee += Drive.broker_fee_percent * account.interest_earned()
        return total_fee

## Concrete example | Decompose conditional

## What is perpose of account checking in line `25-26` ?
```python
if account.account_type == Account.premium or \
    account.account_type == Account.premium_plus:
```
- Threre are several account type
- Fee only applicable for premium category account

## Let's use `decompose conditional` to make it more readable
Adding a method `is_premium` in `Account` class to check a account typs is premium or not
```python
def is_premium(self):
    if self.account_type == Account.premium or \
        self.account_type == Account.premium_plus:
        return True
    return False
```

In [4]:
class Account:
    '''defination of Account class'''
    # defins flags for 4 different types of account
    standard = 0
    budget = 1
    premium = 2
    premium_plus = 3
    
    def __init__(self, principal, rate, days_active, account_type):
        self.principal = principal
        self.rate = rate
        self.days_active = days_active
        self.account_type = account_type
    
    def interest_earned(self):
        return (self.principal * math.exp(
            self.rate * (self.days_active / 365))) - self.principal
    
    def is_premium(self):
        if self.account_type == Account.premium or \
            self.account_type == Account.premium_plus:
            return True
        return False
        
class Drive:
    broker_fee_percent = 0.0125
    
    def calculate_fee(self, accounts: list) -> int:
        '''calculate fees for accounts'''
        total_fee = 0
        for account in accounts:
            if account.is_premium(): # decompose conditional
                total_fee += Drive.broker_fee_percent * account.interest_earned()
        return total_fee

## Concrete example | Explaining variable

## Do you seen any readability issue in this code ? 
```python
def interest_earned(self):
    return (self.principal * math.exp(
        self.rate * (self.days_active / 365))) - self.principal
```
- Confusing calculation
- Intermediate steps in a single line
- Why division by 365
- Why we subtract by `self.principal`

## Can we improve readability in `interest_earned` method of `Account` class?
- Yes
- Breakdown the complex calculation
- Assing meaning full name to each small calculation

```python
def interest_earned(self):
    years = self.days_active / 365
    compound_interest = self.principal * math.exp(self.rate * years)
    interest = compound_interest - self.principal
    return interest
``` 

In [5]:
class Account:
    '''defination of Account class'''
    # defins flags for 4 different types of account
    standard = 0
    budget = 1
    premium = 2
    premium_plus = 3
    
    def __init__(self, principal, rate, days_active, account_type):
        self.principal = principal
        self.rate = rate
        self.days_active = days_active
        self.account_type = account_type
    
    def interest_earned(self):
        years = self.days_active / 365
        compound_interest = self.principal * math.exp(self.rate * years)
        interest = compound_interest - self.principal
        return interest
    
    def is_premium(self):
        if self.account_type == Account.premium or \
            self.account_type == Account.premium_plus:
            return True
        return False
        
class Drive:
    broker_fee_percent = 0.0125
    
    def calculate_fee(self, accounts: list) -> int:
        '''calculate fees for accounts'''
        total_fee = 0
        for account in accounts:
            if account.is_premium(): # decompose conditional
                total_fee += Drive.broker_fee_percent * account.interest_earned()
        return total_fee

## Let's seen difference between `initial code` and `final refactored code`

In [6]:
class Account:
    '''defination of Account class'''
    # defins flags for 4 different types of account
    standard = 0
    budget = 1
    premium = 2
    premium_plus = 3
    
    def __init__(self, principal, rate, days_active, account_type):
        self.principal = principal
        self.rate = rate
        self.days_active = days_active
        self.account_type = account_type

class Drive:
    def calculate_fee(self, accounts: list) -> int:
        '''calculate fees for accounts'''
        total_fee = 0
        for account in accounts:
            if account.account_type == Account.premium or \
                account.account_type == Account.premium_plus:
                total_fee += 0.0125 * (account.principal * math.exp(
                    account.rate * (account.days_active / 365)) - account.principal)
        return total_fee
    
# We can see thre are 24 lines code in our initial code
# On the other hand our final refactored code has 36 lines of code

# But refactored code is more readiable compare to initial source code









In [7]:
class Account:
    '''defination of Account class'''
    # defins flags for 4 different types of account
    standard = 0
    budget = 1
    premium = 2
    premium_plus = 3
    
    def __init__(self, principal, rate, days_active, account_type):
        self.principal = principal
        self.rate = rate
        self.days_active = days_active
        self.account_type = account_type
    
    def interest_earned(self):
        years = self.days_active / 365
        compound_interest = self.principal * math.exp(self.rate * years)
        interest = compound_interest - self.principal
        return interest
    
    def is_premium(self):
        if self.account_type == Account.premium or \
            self.account_type == Account.premium_plus:
            return True
        return False
        
class Drive:
    broker_fee_percent = 0.0125
    
    def calculate_fee(self, accounts: list) -> int:
        '''calculate fees for accounts'''
        total_fee = 0
        for account in accounts:
            if account.is_premium(): # decompose conditional
                total_fee += Drive.broker_fee_percent * account.interest_earned()
        return total_fee
    

## Conclution
There are some other refactoring criterion,

- [Organizing data](https://www.refactoring.com/catalog/?filter=tags-organizing-data)
- [Moving features](https://www.refactoring.com/catalog/?filter=tags-moving-features)
- [Refactoring apis](https://www.refactoring.com/catalog/?filter=tags-refactoring-apis)
- [Grouping functions](https://www.refactoring.com/catalog/?filter=tags-grouping-function)
- [Dealing with inheritance](https://www.refactoring.com/catalog/?filter=tags-dealing-with-inheritance)

Check the complete catelog with code example in [catalog](https://www.refactoring.com/catalog/)

## References
- [Applied Software Project Management - Chapter - Design and Program](https://www.oreilly.com/library/view/applied-software-project/0596009488/)
- [Refactoring catelog](https://www.refactoring.com/catalog/)
- [Create presentation using raise notebook plagin](https://rise.readthedocs.io/en/stable/installation.html)
- [Tips for slide show in jupyter](https://www.markroepke.me/posts/2019/06/05/tips-for-slideshows-in-jupyter.html)

![Question & Answer](images/qa.jpg)